# Introduction to the point selection via P-greedy

In [ ]:
from utils import plot_graph
from graph_loaders import load_graph
import matplotlib.pyplot as plt
import numpy as np
from approx import GBFGreedy
from kernels import VarSpline, Diffusion
import networkx as nx

### Load a graph

We start by loading a pre-defined graph to be used as an example. 

In [ ]:
# G = load_graph('wbc')
# G = load_graph('sensor2')
# G = load_graph('sensor1'
# G = load_graph('emptyset')
# G = load_graph('2moon')
# G = load_graph('minnesota')
# G = load_graph('rand')
# G = load_graph('rand_sparse')
G = load_graph('bunny')

# G = nx.dorogovtsev_goltsev_mendes_graph(7)
# pos = nx.spectral_layout(G, center=[0.5, 0.5])
# nx.set_node_attributes(G, pos, 'pos')

### Define an optimization set

In this case the focus is on point selection only. This means that we can use all the nodes as a training set, but without the need to have target values `y_train`.

In [ ]:
X_train = np.arange(len(G))

Then, we assign dummy target values.

In [ ]:
y_train = np.random.rand(len(G))

The signal looks as follows. The training nodes are highlighted.

### Define a kernel

We now pick a Graph Basis Function as the kernel that will be used in the approximation. 

In [ ]:
# kernel = VarSpline(G, par=[-1.1, 0.01])
kernel = Diffusion(G, par=[-1])

### Reconstruct the signal

We first initialize the approximant. We set `tol_f=0` (i.e., don't use the residual to terminate the algorithm) and instead stop after `max_iter` points are selected or when the max of the squared power function is below `tol_p`. Here we turn off the regularization (i.e., `reg_par=0`) since we are interested purely in the variance minimization.

In [ ]:
max_iter = 100 # Max number of point to be selected
tol_p = 1e-12  # Tolerance on the max of the squared power function
tol_f = 0      # Tolerance on the residual

model = GBFGreedy(G, kernel=kernel, greedy_type='f_greedy', 
                  reg_par=0, 
                  max_iter=max_iter, tol_p=tol_p, tol_f=tol_f)

We can now fit the approximant to the training data.

In [ ]:
model.fit(X_train, y_train)

### Visualize the selected points and the decay of the power function

We visualize the training history.

In [ ]:
p_max = model.train_hist['p']

We estimate the algebraic rate of decay of the power function.

In [ ]:
tail_size = int(0.3 * len(p_max)) 
nn = np.arange(1, len(p_max) + 1)
coeff_max = np.polyfit(np.log(nn)[-tail_size:], np.log(p_max)[-tail_size:], 1)

In [ ]:
fig = plt.figure(figsize=(7, 5))
ax = fig.gca()
a = ax.loglog(p_max)
ax.loglog(nn[-2*tail_size:], np.exp(coeff_max[1]) * nn[-2*tail_size:] ** coeff_max[0], 
              '--', color=a[0].get_color())
ax.legend(['Max of the power function', '$n^{%2.2f}$' % coeff_max[0]], fontsize=16, loc=(1.1, 0.1))
ax.set_xlabel('Number of nodes', fontsize=16)
for tick in ax.xaxis.get_major_ticks():
    tick.label.set_fontsize(16) 
for tick in ax.yaxis.get_major_ticks():
    tick.label.set_fontsize(16) 
ax.grid(True)
ax.grid(True)

And the selected points. In this case we visualize the power function values as a signal.

In [ ]:
p = model.eval_power_fun(X_train)

In [ ]:
fig = plt.figure(figsize=(7, 5))
ax = fig.gca()
plot_graph(G, ax=ax, values=p, nodelist=model.ctrs_, 
           cb_label='Power function')